In [87]:
import cv2
import os
import glob

In [88]:
input_directory = "data/kagome_raw/01_omote"
output_directory = "data/kagome/01_omote_1024"

In [89]:
# gray scale
def gray_scale(image):
    output = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    return output

In [90]:
# meidan filter
def median(image, ksize=7):    
    output = cv2.medianBlur(image, ksize)
    return output

In [91]:
# CLAHEによるヒストグラム平坦化
def clahe_equalization(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    # BGRからLab色空間に変換
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    # Lチャンネルを分離
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    # CLAHEをLチャンネルに適用
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l_channel_clahe = clahe.apply(l_channel)
    # CLAHE後のLチャンネルと元のa, bチャンネルをマージ
    lab_image_clahe = cv2.merge((l_channel_clahe, a_channel, b_channel))
    # LabからBGR色空間に変換
    bgr_image_clahe = cv2.cvtColor(lab_image_clahe, cv2.COLOR_Lab2BGR)
    return bgr_image_clahe

In [92]:
import cv2
import os
import glob
import numpy as np

def resize_image(input_directory, output_directory, w_size, h_size):
    # 対象となるjpg, png, jpegファイルを再帰的に取得
    image_filepaths = glob.glob(os.path.join(input_directory, '**', '*.jpg'), recursive=True) + \
                      glob.glob(os.path.join(input_directory, '**', '*.JPG'), recursive=True) + \
                      glob.glob(os.path.join(input_directory, '**', '*.png'), recursive=True) + \
                      glob.glob(os.path.join(input_directory, '**', '*.jpeg'), recursive=True)
    
    for filepath in image_filepaths:
        # 入力ファイルのパス
        input_filepath = filepath
        
        # 入力画像の相対パスを取得し、拡張子をjpgに変更
        rel_path = os.path.relpath(filepath, input_directory)
        base, _ = os.path.splitext(rel_path)
        relative_path = base + ".jpg"
        output_filepath = os.path.join(output_directory, relative_path)
        output_dir = os.path.dirname(output_filepath)
        
        # 出力先のディレクトリが存在しない場合は作成
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # 画像を読み込む
        img = cv2.imread(input_filepath)
        if img is None:
            print(f"Error reading {input_filepath}")
            continue
        
        # 画像の幅と高さを取得
        height, width = img.shape[:2]
        
        # ターゲットキャンバス内に収めるためのスケーリング係数を計算
        scale = min(w_size / width, h_size / height)
        new_width = int(width * scale)
        new_height = int(height * scale)
        
        # 画像をリサイズ
        resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
        
        # 黒背景のキャンバスを作成
        canvas = np.full((h_size, w_size, 3), 0, dtype=np.uint8)
        
        # キャンバス中央にリサイズした画像を配置するためのオフセットを計算
        x_offset = (w_size - new_width) // 2
        y_offset = (h_size - new_height) // 2
        
        # キャンバスに画像を貼り付け
        canvas[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_img
        
        # 必要に応じて前処理（グレースケール変換、フィルタ処理など）をここで実施
        # 例:
        # canvas = gray_scale(canvas)
        # canvas = median(canvas)
        # canvas = clahe_equalization(canvas)
        
        # 出力画像を保存
        cv2.imwrite(output_filepath, canvas)
    
    print("Resize Image Success")

In [93]:
# 使用例：入力ディレクトリと出力ディレクトリを指定して画像をリサイズ
resize_image(input_directory, output_directory, 1024, 1024)

Resize Image Success
